<a href="https://colab.research.google.com/github/protontypes/AwesomeCure/blob/main/OpenSustain_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenSustain.tech Topic Modeling

ToDO:
* Create Topic Blacklist
* Hierarchical Topic Modeling: https://maartengr.github.io/BERTopic/getting_started/hierarchicaltopics/hierarchicaltopics.html

Maintainer Tips and Tricks:
* Readme into sentences --> fine grant topics
* Representation Model to have "better" words
* Example to use: https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#keybertinspired --> 10 words per topics -->
* Parse -->GPT4



In [ ]:
!pip install bertopic keybert pandas
!wget https://raw.githubusercontent.com/protontypes/AwesomeCure/main/csv/projects.csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 83.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 124.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.

## Reconfigure CSV Readin

In [ ]:
import pandas as pd
import sys
import csv

maxInt = sys.maxsize

# find the largest possible file size
while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)
df = pd.read_csv("projects.csv", engine='python')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from torch.utils.data import dataloader
import re
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

data = pd.DataFrame(columns=["Sentence", "Readme"])
readmes = [str(doc) for doc in df.readme.tolist()]

sentences_cleaned = []
for index, readme in enumerate(readmes):
  sents = sent_tokenize(re.sub(' +', ' ', BeautifulSoup(readme, "lxml").text.replace(r"\n", "")))
  sentences_cleaned.extend(sents)

  for sentence in sents:
    data.loc[len(data), :] = [sentence, index]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic

# Create your representation model
representation_model = KeyBERTInspired()

# Use the representation model in BERTopic on top of the default pipeline
topic_model = BERTopic(representation_model=representation_model, verbose=True).fit(sentences_cleaned)
data["Topic"] = topic_model.topics_

Batches:   0%|          | 0/992 [00:00<?, ?it/s]

2023-06-28 12:45:59,392 - BERTopic - Transformed documents to Embeddings
2023-06-28 12:46:40,458 - BERTopic - Reduced dimensionality
2023-06-28 12:46:43,943 - BERTopic - Clustered reduced embeddings


In [ ]:
topic_model.get_topic_info().head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12079,-1_python_import_file_dataset,"[python, import, file, dataset, datasets, soft...","[Here, we provide a first example but you can ..."
1,0,366,0_conda_environmentconda_condaforge_anaconda,"[conda, environmentconda, condaforge, anaconda...",[Activate this environment: `conda activate qu...
2,1,335,1_rprofile_rstudio_packages_package,"[rprofile, rstudio, packages, package, github,...",[When it is necessary to add a reference to an...
3,2,334,2_trees_forestry_forests_tree,"[trees, forestry, forests, tree, treels, treei...",[The data enables accurate reporting of tree c...
4,3,330,3_photovoltaics_photovoltaic_pv_pvfactors,"[photovoltaics, photovoltaic, pv, pvfactors, p...",[These features are actively under development...
5,4,249,4_github_statushttpsgithubcomjwcooknaturtagwor...,"[github, statushttpsgithubcomjwcooknaturtagwor...",[[](https://github.com/NREL/EnergyPlus/workflo...
6,5,226,5_contributions_contributionsfrom_contribution...,"[contributions, contributionsfrom, contributio...","[Contributions of any kind welcome!, Contribut..."
7,6,217,6_timeseries_timeseriesclustering_datetime_tim...,"[timeseries, timeseriesclustering, datetime, t...",[Image years correspond to the _end_ period of...
8,7,191,7_dockerfile_docker_dockercompose_dockercompos...,"[dockerfile, docker, dockercompose, dockercomp...",[Run from a Docker Container:#### 1.1 Get the ...
9,8,181,8_netcdf_netcdf4python_netcdf3_netcdf4,"[netcdf, netcdf4python, netcdf3, netcdf4, netc...",[b'Unidata NetCDF Fortran Library=============...


In [ ]:
# # Calculate the topic distributions on a token-level
# topic_distr, topic_token_distr = topic_model.approximate_distribution(docs[:2], calculate_tokens=True)

# # Visualize the token-level distributions
# df = topic_model.visualize_approximate_distribution(docs[1], topic_token_distr[1])
# df


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP

# Prepare embeddings
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)

# Train BERTopic
topic_model = BERTopic().fit(docs, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)


## Preprocessing and Cleanup of dataset

In [ ]:
# Replace nan with empty strings in the readmes
import numpy as np
df = df.replace(np.inf, np.nan)
df_readme = df['readme'].fillna('')
docs = df_readme.to_list()

# Replace nan with empty strings in topics
df_topics = df["topics"].fillna('')
supervised_topics = df_topics.to_list()

# create a subset of the dataset to speedup the processing
docs_subset = docs

In [ ]:
# Create a flat list of all userdefined topics
supervised_topics_flat = []
for word in supervised_topics:
    word = word.split(",")
    supervised_topics_flat.extend(word)
supervised_topics_flat = [x for x in supervised_topics_flat if x]

In [ ]:
supervised_topics_flat

['solar-energy',
 'python',
 'renewable-energy',
 'renewables',
 'photovoltaic',
 'solar-energy',
 'renewable-energy',
 'python',
 'bifacial',
 'solar',
 'pandas',
 'energy',
 'irradiance',
 'photovoltaic',
 'pv',
 'electricity',
 'ninja',
 'numpy',
 'scipy',
 'python',
 'solar',
 'photovoltaic',
 'machine-learning',
 'neural-network',
 'data-processing',
 'python',
 'matlab',
 'tensorflow',
 'photovoltaic',
 'solar-energy',
 'solar-cells',
 'machine-learning',
 'computer-vision',
 'python',
 'photovoltaics',
 'raytracing',
 'optics',
 'energy',
 'radiance',
 'bifacial',
 'photovoltaics',
 'renewable-energy',
 'renewables',
 'gui',
 'photovoltaic',
 'semiconductor',
 'solar-cells',
 'python',
 'hacktoberfest',
 'solax',
 'solar',
 'photovoltaic',
 'home-automation',
 'iot',
 'raspberry-pi',
 'engineering',
 'energy',
 'solar',
 'thermodynamics',
 'optimisation',
 'simulation',
 'modelica',
 'modelica-library',
 'pvoutput',
 'python',
 'python-library',
 'solar',
 'nowcasting',
 'nowcas

In [ ]:
# Define a list of blacklist topics and remove them from the topic list
topic_black_list = ["install","data","python","using","use"]
topic_black_list_set = set(topic_black_list)
supervised_topics_flat_cleaned = [x for x in supervised_topics_flat if x not in topic_black_list_set]

In [ ]:
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer


# Extract keywords
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(docs)

vocabulary = [k[0] for keyword in keywords for k in keyword]
vocabulary = list(set(vocabulary))
vocabulary = [str(i) for i in vocabulary]

vectorizer_model= CountVectorizer(vocabulary=vocabulary,stop_words="english")


In [ ]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer

ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)

topic_model = BERTopic(ctfidf_model=ctfidf_model, min_topic_size=10)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,1300,0_the_and_to_of,"[the, and, to, of, for, in, is, data, with, on]",[b'# GlobalEnergyGIS.jl\n\nAutomatic generatio...
1,1,138,1____,"[, , , , , , , , , ]","[, , ]"
2,2,12,2____,"[, , , , , , , , , ]","[, , ]"


In [ ]:
subset_training = topic_model.get_document_info(docs_subset)

In [ ]:
subset_training

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,"b'<img src=""docs/sphinx/source/_images/pvlib_l...",0,0_the_and_to_of,"[the, and, to, of, for, in, is, data, with, on]",[b'# GlobalEnergyGIS.jl\n\nAutomatic generatio...,the - and - to - of - for - in - is - data - w...,1.000000,False
1,b'pvfactors: irradiance modeling made simple\n...,0,0_the_and_to_of,"[the, and, to, of, for, in, is, data, with, on]",[b'# GlobalEnergyGIS.jl\n\nAutomatic generatio...,the - and - to - of - for - in - is - data - w...,1.000000,False
2,"b""[![Master branch build status](https://img.s...",0,0_the_and_to_of,"[the, and, to, of, for, in, is, data, with, on]",[b'# GlobalEnergyGIS.jl\n\nAutomatic generatio...,the - and - to - of - for - in - is - data - w...,1.000000,False
3,b'PVMismatch\n==========\n\nAn explicit IV & P...,0,0_the_and_to_of,"[the, and, to, of, for, in, is, data, with, on]",[b'# GlobalEnergyGIS.jl\n\nAutomatic generatio...,the - and - to - of - for - in - is - data - w...,1.000000,False
4,"b'<img src=""./docs/sphinx/source/_images/logo_...",0,0_the_and_to_of,"[the, and, to, of, for, in, is, data, with, on]",[b'# GlobalEnergyGIS.jl\n\nAutomatic generatio...,the - and - to - of - for - in - is - data - w...,0.936615,False
...,...,...,...,...,...,...,...,...
1445,,1,1____,"[, , , , , , , , , ]","[, , ]",- - - - - - - - -,0.511595,True
1446,b'# Awesome Sentinel [![Awesome](https://cdn.r...,0,0_the_and_to_of,"[the, and, to, of, for, in, is, data, with, on]",[b'# GlobalEnergyGIS.jl\n\nAutomatic generatio...,the - and - to - of - for - in - is - data - w...,1.000000,False
1447,"b""# geospatial-data-catalogs\n\n[![Colab](http...",0,0_the_and_to_of,"[the, and, to, of, for, in, is, data, with, on]",[b'# GlobalEnergyGIS.jl\n\nAutomatic generatio...,the - and - to - of - for - in - is - data - w...,1.000000,False
1448,,1,1____,"[, , , , , , , , , ]","[, , ]",- - - - - - - - -,0.860171,True
